In [2]:
using JuMP;
using DataFrames;
using Gurobi;
using GLPK;
using MaxEntChemostat2018;

## Toy Model

In [72]:
rxns = DataFrame();
rxns[:id] = ["rxn1","rxn2","rxn3","rxn4","At", "Bt", "Ft", 
    "EX_A_e","EX_B_e","EX_F_e","Obj"];
let M = 10^6
    ###########  r1  r2  r3  r4  At  Bt  Ft ExA  ExB ExF Obj ##
    rxns[:lb] = [0, -M,  0, -M, -M, -M, -M, -10, -M,  0,  0];
    rxns[:ub] = [M,  M,  M,  M,  M,  M,  M,  M,   M,  M,  M];
#     rxns[:lb] = [0, -M,  0, -M, -M, -M, -M, -10, -M,  0,  0];
#     rxns[:ub] = [M,  M,  M,  M,  M,  M,  M,  M,   M,  M,  M];
end

rxns

,id,lb,ub
1,rxn1,0,1000000
2,rxn2,-1000000,1000000
3,rxn3,0,1000000
4,rxn4,-1000000,1000000
5,At,-1000000,1000000
6,Bt,-1000000,1000000
7,Ft,-1000000,1000000
8,EX_A_e,-10,1000000
9,EX_B_e,-1000000,1000000
10,EX_F_e,0,1000000


In [73]:
mets = DataFrame([["A_e","B_e","A_c","B_c","C_c","D_c",
            "E_c","F_c","F_e"]],[:id])

,id
1,A_e
2,B_e
3,A_c
4,B_c
5,C_c
6,D_c
7,E_c
8,F_c
9,F_e


In [74]:
###  r1  r2  r3  r4  At  Bt  Ft ExA ExB ExF Obj ##
S = [0   0   0   0   1   0   0  -1   0   0   0  ## A_e
     0   0   0   0   0   1   0   0  -1   0   0  ## B_e
    -1   0   0   0  -1   0   0   0   0   0   0  ## A_c
    -1   0   0   0   0  -1   0   0   0   0   0  ## B_c
     1  -1  -1   0   0   0   0   0   0   0   0  ## C_c
     0   0   1   0   0   0   0   0   0   0  -1  ## D_c
     0   0   1  -1   0   0   0   0   0   0   0  ## E_c
     0   2   0   1   0   0  -1   0   0   0   0  ## F_c
     0   0   0   0   0   0   1   0   0  -1   0] ## F_e

9×11 Array{Int64,2}:
  0   0   0   0   1   0   0  -1   0   0   0
  0   0   0   0   0   1   0   0  -1   0   0
 -1   0   0   0  -1   0   0   0   0   0   0
 -1   0   0   0   0  -1   0   0   0   0   0
  1  -1  -1   0   0   0   0   0   0   0   0
  0   0   1   0   0   0   0   0   0   0  -1
  0   0   1  -1   0   0   0   0   0   0   0
  0   2   0   1   0   0  -1   0   0   0   0
  0   0   0   0   0   0   1   0   0  -1   0

## EColi Model

In [3]:
S, mets, rxns = MaxEntChemostat2018.FBA.load_ecoli();

In [62]:
@assert all(size(S) .== (size(mets,1), size(rxns,1)))
@assert allunique(rxns[:id])
@assert allunique(mets[:id])

println("Bulding Model")
model = Model();
setsolver(model, Gurobi.GurobiSolver());

#Variables:  The fluxes
println("Setting Variables");
vars = Vector{JuMP.Variable}();
for r in 1:size(rxns,1)
    var = @variable(model, basename = rxns[:id][r])
#     setcategory(var, :SemiCont);
    push!(vars, var);
    
    r % 100 == 0 && println("v: $r / $(size(rxns,1))");
    
end
obj = vars[end];

#Constraints
#upper and lower bounds
println("Setting Bounds Constraints")
for r in 1:length(vars)
    @constraint(model, vars[r] >= rxns[:lb][r]);
    @constraint(model, vars[r] <= rxns[:ub][r]);
    r % 100 == 0 && println("v: $r / $(size(rxns,1))");
end
obj = vars[13];

#S*v = 0
println("Setting Steady State Constraint")
for m in 1:size(mets,1)
    @constraint(model,  S[m,:]' * vars == 0)
end

#Cost contraints Sum(an*r + ap*r)
let C = 0.1
    @constraint(model, sum(((rxns[:an] + rxns[:ap])/2)'*vars) <= C)
end

#objective
println("Setting Objective")
@objective(model, Max, obj)

println(model)

Bulding Model
Setting Variables
Setting Bounds Constraints
Setting Steady State Constraint
Setting Objective
Max Biomass_Ecoli_core_w/GAM
Subject to
 ACALD ≥ -1000
 ACALD ≤ 1000
 ACALDt ≥ -1000
 ACALDt ≤ 1000
 ACKr ≥ -1000
 ACKr ≤ 1000
 ACONTa ≥ -1000
 ACONTa ≤ 1000
 ACONTb ≥ -1000
 ACONTb ≤ 1000
 ACt2r ≥ -1000
 ACt2r ≤ 1000
 ADK1 ≥ -1000
 ADK1 ≤ 1000
 AKGDH ≥ 0
 AKGDH ≤ 1000
 AKGt2r ≥ -1000
 AKGt2r ≤ 1000
 ALCD2x ≥ -1000
 ALCD2x ≤ 1000
 ATPM ≥ 0
 ATPM ≤ 1000
 ATPS4r ≥ -1000
 ATPS4r ≤ 1000
 Biomass_Ecoli_core_w/GAM ≥ 0
 Biomass_Ecoli_core_w/GAM ≤ 1000
 CO2t ≥ -1000
 CO2t ≤ 1000
 CS ≥ 0
 CS ≤ 1000
 CYTBD ≥ 0
 CYTBD ≤ 1000
 D_LACt2 ≥ -1000
 D_LACt2 ≤ 1000
 ENO ≥ -1000
 ENO ≤ 1000
 ETOHt2r ≥ -1000
 ETOHt2r ≤ 1000
 EX_ac(e) ≥ 0
 EX_ac(e) ≤ 1000
 EX_acald(e) ≥ 0
 EX_acald(e) ≤ 1000
 EX_akg(e) ≥ 0
 EX_akg(e) ≤ 1000
 EX_co2(e) ≥ -1000
 EX_co2(e) ≤ 1000
 EX_etoh(e) ≥ 0
 EX_etoh(e) ≤ 1000
 EX_for(e) ≥ 0
 EX_for(e) ≤ 1000
 EX_fru(e) ≥ 0
 EX_fru(e) ≤ 1000
 EX_fum(e) ≥ 0
 EX_fum(e) ≤ 1000
 EX_glc(

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] uuid4(::MersenneTwister) at ./deprecated.jl:57
 [3] msg_header at /Users/Pereiro/.julia/v0.6/IJulia/src/msg.jl:18 [inlined]
 [4] msg_pub(::IJulia.Msg, ::String, ::Dict{String,String}, ::Dict{String,Any}) at /Users/Pereiro/.julia/v0.6/IJulia/src/msg.jl:30 (repeats 2 times)
 [5] send_stream(::String) at /Users/Pereiro/.julia/v0.6/IJulia/src/stdio.jl:172
 [6] send_stdio(::String) at /Users/Pereiro/.julia/v0.6/IJulia/src/stdio.jl:130
 [7] (::Base.##302#303{IJulia.#send_stdout,Timer})() at ./event.jl:436
while loading In[62], in expression starting on line 44
Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] uuid4(::MersenneTwister) at ./deprecated.jl:57
 [3] msg_header at /Users/Pereiro/.julia/v0.6/IJulia/src/msg.jl:18 [inlined]
 [4] msg_pub(::IJulia.Msg, ::String, ::Dict{String,String}, ::Dict{String,Any}) at /Users/Pereiro/.julia/v0.6/IJulia/src/msg.jl:30 (repeats 2 times)
 [5] send_stream(::Str

## Solving

In [63]:
JuMP.solve(model);

Academic license - for non-commercial use only
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Optimize a model with 263 rows, 95 columns and 645 nonzeros
Coefficient statistics:
  Matrix range     [1e-07, 6e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 1e+03]
Presolve removed 233 rows and 43 columns
Presolve time: 0.00s
Presolved: 30 rows, 52 columns, 256 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.3682659e+01   3.581450e+03   0.000000e+00      0s
      31    3.0712097e-02   0.000000e+00   0.000000e+00      0s

Solved in 31 iterations and 0.00 seconds
Optimal objective  3.071209689e-02


In [64]:
DataFrame([string.(vars) Float64.(getvalue.(vars))])

,x1,x2
1,ACALD,-0.953151
2,ACALDt,0.0
3,ACKr,0.0
4,ACONTa,0.121144
5,ACONTb,0.121144
6,ACt2r,0.0
7,ADK1,0.0
8,AKGDH,0.0
9,AKGt2r,0.0
10,ALCD2x,-0.953151


In [162]:
let nz_index = Vector{Int}()
    for ri in 1:length(vars)
        if getvalue(vars[ri]) != 0.0
            push!(nz_index, ri);
        end
    end
    
    println("Rxns $(length(nz_index)) / $(size(rxns,1))");
    global nz_rxns = rxns[nz_index,:];
    global nz_S = S[:,nz_index];
    
    nz_index = Vector{Int}();
    for mi in 1:size(mets,1)
        if length(nonzeros(nz_S[mi,:])) != 0
           push!(nz_index, mi);
        end
    end
    global nz_mets = mets[nz_index,:];
    global nz_S = nz_S[nz_index,:];
    println("Mets $(length(nz_index)) / $(size(mets,1))");
end
(nz_S, nz_mets, nz_rxns)

Rxns 48 / 95
Mets 52 / 72


(
  [4 ,  1]  =  -1.0
  [5 ,  1]  =  1.0
  [13,  1]  =  -1.0
  [29,  1]  =  1.0
  [33,  1]  =  -1.0
  [34,  1]  =  1.0
  [6 ,  2]  =  1.0
  [10,  2]  =  -1.0
  [27,  2]  =  1.0
  [6 ,  3]  =  -1.0
  ⋮
  [36, 45]  =  1.0
  [21, 46]  =  1.0
  [48, 46]  =  -1.0
  [50, 46]  =  1.0
  [52, 46]  =  -1.0
  [15, 47]  =  -1.0
  [18, 47]  =  1.0
  [21, 47]  =  1.0
  [52, 47]  =  -1.0
  [14, 48]  =  -1.0
  [21, 48]  =  1.0, 52×6 DataFrames.DataFrame
│ Row │ id     │ y       │ e    │ L       │ V      │ c    │
├─────┼────────┼─────────┼──────┼─────────┼────────┼──────┤
│ 1   │ 13dpg  │ 0.0     │ 0.0  │ 0.0     │ 0.0    │ 0.0  │
│ 2   │ 2pg    │ 0.0     │ 0.0  │ 0.0     │ 0.0    │ 0.0  │
│ 3   │ 3pg    │ 1.496   │ 0.0  │ 0.0     │ 0.0    │ 0.0  │
│ 4   │ acald  │ 0.0     │ 0.0  │ 0.0     │ 0.0    │ 0.0  │
│ 5   │ accoa  │ 3.7478  │ 0.0  │ 0.0     │ 0.0    │ 0.0  │
│ 6   │ acon-C │ 0.0     │ 0.0  │ 0.0     │ 0.0    │ 0.0  │
│ 7   │ adp    │ -59.81  │ -1.0 │ 0.0     │ 0.0    │ 0.0  │
│ 8   │ akg    │ -

In [140]:
nonzeros(nz_S[4,:])

0-element Array{Float64,1}

In [158]:
S[((1:3);(6:end)),:]

67×95 SparseMatrixCSC{Float64,Int64} with 349 stored entries:
  [3 ,  1]  =  -1.0
  [5 ,  1]  =  1.0
  [16,  1]  =  -1.0
  [38,  1]  =  1.0
  [45,  1]  =  -1.0
  [46,  1]  =  1.0
  [3 ,  2]  =  1.0
  [4 ,  2]  =  -1.0
  [1 ,  3]  =  -1.0
  [7 ,  3]  =  1.0
  ⋮
  [48, 92]  =  1.0
  [28, 93]  =  1.0
  [61, 93]  =  -1.0
  [63, 93]  =  1.0
  [67, 93]  =  -1.0
  [18, 94]  =  -1.0
  [21, 94]  =  1.0
  [28, 94]  =  1.0
  [67, 94]  =  -1.0
  [17, 95]  =  -1.0
  [28, 95]  =  1.0